# Imports

In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import models
import loader
import training
import metrics

# GPU

In [2]:
device = torch.device("cuda")

# Data Loading

In [3]:
mnist_data_path = './train-images-idx3-ubyte'
mnist_label_path = './train-labels-idx1-ubyte'

cifar_data_path = './cifar-10-batches-py'

In [4]:
#5-fold generation
mnist_data = loader.MNIST(mnist_data_path, mnist_label_path, 5)
mnist_splits = mnist_data.splits

cifar_data = loader.CIFAR(cifar_data_path, 5)
cifar_splits = cifar_data.splits

In [5]:
print("MNIST STATS")
print("Number of splits:", len(mnist_splits))
print("Number of segments per split (train, val, test):", len(mnist_splits[0]))
print("Info per segment (data, labels):", len(mnist_splits[0][0]))
print("Size of segement (num examples):", len(mnist_splits[0][0][0]))
print()
print("CIFAR STATS")
print("Number of splits:", len(cifar_splits))
print("Number of segments per split (train, val, test):", len(cifar_splits[0]))
print("Info per segment (data, labels):", len(cifar_splits[0][0]))
print("Size of segement (num examples):", len(cifar_splits[0][0][0]))

MNIST STATS
Number of splits: 5
Number of segments per split (train, val, test): 3
Info per segment (data, labels): 2
Size of segement (num examples): 48000

CIFAR STATS
Number of splits: 5
Number of segments per split (train, val, test): 3
Info per segment (data, labels): 2
Size of segement (num examples): 40000


# Model

In [6]:
mnist_net = models.VanillaCNN(in_channels=1)
cifar_net = models.VanillaCNN(in_channels=3)

# Training and Evaluation

## MNIST

In [7]:
mnist_trainer = training.Trainer(mnist_data, device)

In [ ]:
mnist_trainer.train(mnist_net, 20, 1e-3, 128, 8, verbose=False, num_loss=3)

[Fold 1] Epoch:1 Training Acc:0.3649375
[Fold 1] Epoch:1 Validation Acc:0.6368333333333334
[Fold 1] Epoch:2 Training Acc:0.6105208333333333
[Fold 1] Epoch:2 Validation Acc:0.8205
[Fold 1] Epoch:3 Training Acc:0.7319583333333334
[Fold 1] Epoch:3 Validation Acc:0.9123333333333333
[Fold 1] Epoch:4 Training Acc:0.788625
[Fold 1] Epoch:4 Validation Acc:0.929
[Fold 1] Epoch:5 Training Acc:0.815625
[Fold 1] Epoch:5 Validation Acc:0.9348333333333333
[Fold 1] Epoch:6 Training Acc:0.8329375
[Fold 1] Epoch:6 Validation Acc:0.9468333333333333
[Fold 1] Epoch:7 Training Acc:0.8415833333333333
[Fold 1] Epoch:7 Validation Acc:0.9493333333333334
[Fold 1] Epoch:8 Training Acc:0.8523541666666666
[Fold 1] Epoch:8 Validation Acc:0.9495
[Fold 1] Epoch:9 Training Acc:0.8580416666666667
[Fold 1] Epoch:9 Validation Acc:0.9535
[Fold 1] Epoch:10 Training Acc:0.8646458333333333
[Fold 1] Epoch:10 Validation Acc:0.9578333333333333
[Fold 1] Epoch:11 Training Acc:0.8674375
[Fold 1] Epoch:11 Validation Acc:0.960333333

[Fold 5] Epoch:10 Validation Acc:0.9631666666666666
[Fold 5] Epoch:11 Training Acc:0.8998541666666666
[Fold 5] Epoch:11 Validation Acc:0.9623333333333334
[Fold 5] Epoch:12 Training Acc:0.903125
[Fold 5] Epoch:12 Validation Acc:0.9643333333333334
[Fold 5] Epoch:13 Training Acc:0.9029583333333333
[Fold 5] Epoch:13 Validation Acc:0.9661666666666666
[Fold 5] Epoch:14 Training Acc:0.9088125
[Fold 5] Epoch:14 Validation Acc:0.9651666666666666
[Fold 5] Epoch:15 Training Acc:0.911625
[Fold 5] Epoch:15 Validation Acc:0.9685
[Fold 5] Epoch:16 Training Acc:0.9124791666666666


In [ ]:
mnist_trainer.test(mnist_net, 128, 12)

In [ ]:
mnist_plotter = metrics.Plotter(mnist_trainer.val_metrics)

In [ ]:
mnist_plotter.plotLoss("Loss")

In [ ]:
mnist_plotter.plotLoss("Loss", mnist_trainer.metrics())

In [ ]:
mnist_plotter.plotAccuracy("Acc")

## CIFAR

In [ ]:
cifar_trainer = training.Trainer(cifar_data, device)

In [ ]:
cifar_trainer.train(cifar_net, 1, 1e-3, 32, 8, verbose=False, num_loss=3)

In [ ]:
cifar_trainer.test(cifar_net, 128, 12)